In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import pandas as pd
import sklearn
from tqdm.auto import tqdm

import scipy.stats
import sklearn.neighbors

In [3]:
#dataset_root_dir = 'dhs_train'

In [4]:
train_df = pd.read_csv("water_train_clean.csv")
train_df['urban_bi'] = train_df['urban'].map({'R': 0, 'U': 1})
#train_df.dropna(inplace=True)

In [5]:
test_df = pd.read_csv("test_masked_new.csv")
test_df['urban_bi'] = test_df['urban'].map({'R': 0, 'U': 1})
#test_df.dropna(inplace=True)

In [6]:
test_df

,Unnamed: 0,index,NDVI,AWEI,NDWI,MNDWI,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,cluster_id,adm1dhs,urban,path,new_path,urban_bi
0,0,0,0.281900,-0.315224,0.023095,-0.170419,IA-2015-7-00010009,IA,2015,9.220903,92.781530,22.0,2.721812,10009,1,R,dhs_valid/IA-2015-7-00010009.npz,./dhs_test/IA-2015-7-00010009.npz,0
1,1,1,NaN,-0.566491,NaN,NaN,IA-2015-7-00010011,IA,2015,7.028410,93.883430,20.0,2.287279,10011,1,R,dhs_valid/IA-2015-7-00010011.npz,./dhs_test/IA-2015-7-00010011.npz,0
2,2,2,0.345973,-0.050747,-0.032956,-0.389066,IA-2015-7-00010017,IA,2015,12.371448,92.783665,22.0,0.677109,10017,1,R,dhs_valid/IA-2015-7-00010017.npz,./dhs_test/IA-2015-7-00010017.npz,0
3,3,3,0.290802,-0.226033,-0.010008,-0.262943,IA-2015-7-00010044,IA,2015,11.727304,92.719257,21.0,1.793683,10044,1,R,dhs_valid/IA-2015-7-00010044.npz,./dhs_test/IA-2015-7-00010044.npz,0
4,4,4,0.579753,-0.672463,-0.302869,-0.564124,IA-2015-7-00010060,IA,2015,9.185310,92.777645,22.0,2.758168,10060,1,R,dhs_valid/IA-2015-7-00010060.npz,./dhs_test/IA-2015-7-00010060.npz,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2670,2670,2670,0.132755,-0.357653,-0.111590,-0.366728,IA-2015-7-00360403,IA,2015,17.410860,78.558810,19.0,2.823787,360403,36,U,dhs_valid/IA-2015-7-00360403.npz,./dhs_test/IA-2015-7-00360403.npz,1
2671,2671,2671,0.126751,-0.426303,-0.138956,-0.403602,IA-2015-7-00360454,IA,2015,17.450098,78.360138,22.0,1.990244,360454,36,R,dhs_valid/IA-2015-7-00360454.npz,./dhs_test/IA-2015-7-00360454.npz,0
2672,2672,2672,0.240977,-0.916248,-0.196299,-0.431396,IA-2015-7-00360474,IA,2015,17.973616,79.597408,20.0,2.633020,360474,36,U,dhs_valid/IA-2015-7-00360474.npz,./dhs_test/IA-2015-7-00360474.npz,1
2673,2673,2673,0.035671,-0.238419,-0.007187,-0.356937,IA-2015-7-00360476,IA,2015,18.956406,77.934361,21.0,0.429279,360476,36,R,dhs_valid/IA-2015-7-00360476.npz,./dhs_test/IA-2015-7-00360476.npz,0


In [7]:
display(train_df.head())

,Unnamed: 0.2,Unnamed: 0.1,index,NDVI,AWEI,NDWI,MNDWI,Unnamed: 0,DHSID_EA,cname,...,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1dhs,urban,path,urban_bi
0,0,0,0,0.150913,0.137764,0.143159,-0.336063,1,IA-2015-7-00010027,IA,...,92.883517,22,-2.836691,22,1.136364,10027,1,R,dhs_train/IA-2015-7-00010027.npz,0
1,1,1,1,-0.061623,0.519389,0.156040,-0.299680,2,IA-2015-7-00020670,IA,...,81.329535,21,0.732851,21,2.523810,20670,2,R,dhs_train/IA-2015-7-00020670.npz,0
2,2,2,2,0.175093,-0.350465,-0.091609,-0.447495,3,IA-2015-7-00020924,IA,...,80.525736,21,1.207174,21,3.000000,20924,2,R,dhs_train/IA-2015-7-00020924.npz,0
3,3,3,3,0.097361,0.192039,0.058564,-0.325924,4,IA-2015-7-00020926,IA,...,82.077392,21,-1.034582,21,3.000000,20926,2,R,dhs_train/IA-2015-7-00020926.npz,0
4,4,4,4,0.209335,-0.463782,-0.135953,-0.358507,5,IA-2015-7-00030004,IA,...,96.010830,19,-0.106301,19,3.000000,30004,3,R,dhs_train/IA-2015-7-00030004.npz,0


## new variables

In [8]:
feature_cols = ['AWEI']
label_cols = ['water_index']

In [9]:
k = 10
knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='uniform', algorithm='auto')

In [10]:
train_X = train_df[feature_cols]
train_Y = train_df[label_cols]
test_X = test_df[feature_cols]

In [11]:
train_X

,AWEI
0,0.137764
1,0.519389
2,-0.350465
3,0.192039
4,-0.463782
...,...
14012,-0.933989
14013,-0.469491
14014,-0.148868
14015,-0.653511


In [12]:
knn.fit(train_X, train_Y)
preds = knn.predict(test_X)

In [13]:
solution_df = test_df.copy()
solution_df["water_index"] = preds

In [14]:
filtered_solution_df = solution_df[["DHSID_EA", "water_index"]]
filtered_solution_df.to_csv("solution.csv", index=False)

In [15]:
filtered_solution_df

,DHSID_EA,water_index
0,IA-2015-7-00010009,3.653420
1,IA-2015-7-00010011,4.223241
2,IA-2015-7-00010017,3.790801
3,IA-2015-7-00010044,4.090043
4,IA-2015-7-00010060,4.610606
...,...,...
2670,IA-2015-7-00360403,3.680087
2671,IA-2015-7-00360454,4.225974
2672,IA-2015-7-00360474,4.213045
2673,IA-2015-7-00360476,3.550059
